In [1]:
# importing libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd


import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### Scrape and transform the data about Toronto neighbrhood into a dataframe ###

In [2]:
# specify the url
post_codes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# query the website and return the html to the variable ‘page’
page = requests.get(post_codes, timeout=5)

In [4]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page.content, 'html.parser')

### Creating the dataframe by droping the cells with "not assigned" borough and combining borough listed twice ###

In [5]:
code_table = soup.find('table')
code_rows = code_table.findAll('tr')
columns=['Postcode', 'Borough', 'Neighbourhood']
df_codes = pd.DataFrame(columns=columns)
for idx, val in enumerate(code_rows):
    code_cells = val.findAll('td')
    df_list = []
    for idx, val in enumerate(code_cells):
        df_list.append(val.text.rstrip())
    if(int(len(df_list)) > 0):
        if(df_list[1] != "Not assigned"):
            if(df_list[2] == "Not assigned"):
                df_list[2] = df_list[1]

            df_dic={columns[0]: df_list[0], columns[1]: df_list[1], columns[2]: df_list[2]}
            df_codes = df_codes.append(df_dic, ignore_index=True)

df_codes = df_codes.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))                

### shape of our dataframe ###

In [6]:
df_codes.shape

(103, 3)